In [1]:
!pip -q install sentence_transformers faiss-cpu pandas rapidfuzz cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.3/295.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 81.8 MB/s eta 0:00:00


In [2]:
# 경로/설정
import os, json, random, numpy as np, pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional
from google.colab import drive
drive.mount('/content/drive')

EMB_FILE = "/content/drive/MyDrive/data/output_chunks_with_embeddings.json"  # 임베딩
QA_PATH  = "/content/drive/MyDrive/data/real_estate_tax_QA.json"            # QA

TOPK_RETR   = 30   # 리트리버 후보 개수(k1)
TOPK_FINAL  = 5    # 리랭크 최종 컨텍스트(k2)
MAX_LEN_CE  = 512  # Cross-Encoder 입력 길이
SEED        = 42
random.seed(SEED); np.random.seed(SEED)

# Cohere API 키 (코랩 환경에서 불러오기)
try:
    from google.colab import userdata
    COHERE_KEY = os.getenv("COHERE_KEY") or userdata.get("COHERE_KEY")
except Exception:
    COHERE_KEY = os.getenv("COHERE_KEY")

import cohere
co_client = cohere.Client(COHERE_KEY) if COHERE_KEY else None
print("Cohere API:", "OK" if co_client else "NOT SET")

Mounted at /content/drive
Cohere API: OK


In [3]:
import json, numpy as np

with open(EMB_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [d.get("text","") for d in data if isinstance(d.get("text"), str)]
embs  = np.asarray([d["embedding"] for d in data if isinstance(d.get("embedding"), list)], dtype="float32")
meta  = [{
    "doc_id": d.get("doc_id"),
    "chunk_index": d.get("chunk_index"),
    "filename": d.get("filename"),
    "folder": d.get("folder"),
} for d in data]

print(f"청크 수: {len(texts)} | 임베딩 shape: {embs.shape}")
print("샘플:", texts[0][:150].replace("\n"," "))

청크 수: 1057 | 임베딩 shape: (1057, 1024)
샘플: 양도소득세부과처분취소 [수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소] 【판시사항】 양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에 있어 주상복합건축물의 건물 외벽 내부에 있는 발코니 면적을 전용면적에 포함시켜야 하는지 


In [5]:
# QA 파일 로드
with open(QA_PATH, "r", encoding="utf-8") as f:
    try:
        qa_raw = json.load(f)
    except json.JSONDecodeError:
        f.seek(0)
        qa_raw = [json.loads(line) for line in f if line.strip()]

# 필드 추출 (데이터 구조 그대로 유지)
questions = [x.get("question") for x in qa_raw]
answers   = [x.get("ground_truth") for x in qa_raw]
contexts  = [x.get("ground_truth_contexts", []) for x in qa_raw]

# 유효한 질문만 유지
qa = [(q, a, c) for q, a, c in zip(questions, answers, contexts) if isinstance(q, str) and q.strip()]

print(f"QA 수: {len(qa)}")
if qa:
    q0, a0, c0 = qa[0]
    print("샘플 질문:", q0)
    if a0:
        print("샘플 정답:", (a0[:150] + "...") if len(a0) > 150 else a0)
    print("샘플 근거 문장:", c0[:3])

QA 수: 50
샘플 질문: 종합부동산세에서 공제할 재산세는 어떻게 산정해야 하나요?
샘플 정답: 종합부동산세에서 공제할 재산세는 공시가격에서 기준금액을 뺀 금액에 재산세 공정시장가액비율과 종합부동산세 공정시장가액비율 가운데 작은 비율을 곱하고, 이에 재산세 세율을 적용해 계산해야 합니다.
샘플 근거 문장: ['종부세 납부세액 계산시 공제되는 재산세액을 (공시가격-과세기준금액)×재산세와 종합부동산세의 공정시장가액비율 중 적은 비율×재산세율의 산식에 따라 산정하여야 할 것임']


In [6]:
# 각 청크의 고유 ID (doc_id#chunk_index)
ids = [f"{m.get('doc_id')}#{m.get('chunk_index')}" for m in meta]

# 빠른 참조용 매핑
id2text = {i: t for i, t in zip(ids, texts)}
id2meta = {i: m for i, m in zip(ids, meta)}

print("예시 ID:", ids[0])
print("예시 텍스트:", id2text[ids[0]][:120].replace("\n"," "), "...")


예시 ID: 판례_136542#0
예시 텍스트: 양도소득세부과처분취소 [수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소] 【판시사항】 양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에 있어 주상복합건축물의 건물 외벽 내 ...


In [7]:
import faiss
from sentence_transformers import SentenceTransformer
import torch

# 내적용 정규화 + 인덱스
def _normalize(mat):
    n = np.linalg.norm(mat, axis=1, keepdims=True) + 1e-12
    return mat / n

dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(_normalize(embs).astype("float32"))
print("FAISS index size:", index.ntotal)

# e5 쿼리 임베딩
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
_e5 = None
def e5_query_embed(q: str):
    global _e5
    if _e5 is None:
        _e5 = SentenceTransformer("intfloat/multilingual-e5-base", device=DEVICE)# 768 dim
        _e5.max_seq_length = 512
    v = _e5.encode([f"query: {q.strip()}"], normalize_embeddings=True, show_progress_bar=False)[0]
    return v.astype("float32")

# Dense 검색 → 후보 ID/점수
def retrieve_candidates(query: str, topk: int = TOPK_RETR):
    qv = e5_query_embed(query)
    qn = (qv / (np.linalg.norm(qv) + 1e-12)).astype("float32")[None, :]
    D, I = index.search(qn, topk)
    cand_ids  = [ids[i] for i in I[0]]
    cand_scos = [float(s) for s in D[0]]
    return cand_ids, cand_scos

FAISS index size: 1057


#**로컬 리랭커(BGE, MARCO, Jina)와 Cohere 리랭커**

In [10]:
from sentence_transformers import CrossEncoder
import numpy as np
import os, cohere

# 리랭커 카탈로그 (로컬 / Cohere만 API 필요)
LOCAL_RERANKERS = {
    # BGE
    "bge-reranker-base":   "BAAI/bge-reranker-base",
    "bge-reranker-large":  "BAAI/bge-reranker-large",
    "bge-reranker-v2-m3":  "BAAI/bge-reranker-v2-m3",

    # MS MARCO Cross-Encoder
    "MiniLM-msmarco":        "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "MiniLM-msmarco-L12":    "cross-encoder/ms-marco-MiniLM-L-12-v2",
    "distilroberta-msmarco": "cross-encoder/ms-marco-distilroberta-base-v2",
    "electra-msmarco":       "cross-encoder/ms-marco-electra-base",
    "tinybert-msmarco":      "cross-encoder/ms-marco-TinyBERT-L-2-v2",

    # Jina (공개된 모델)
    "jina-v2-multilingual":  "jinaai/jina-reranker-v2-base-multilingual",
    "jina-v1-en":            "jinaai/jina-reranker-v1-base-en",
}

# 프롬프트 구성
USE_META_HEADER = True
def _header(m): return f"doc:{m.get('doc_id')} | file:{m.get('filename')} | chunk:{m.get('chunk_index')}"
def _pair(query: str, did: str):
    txt = id2text[did]
    if USE_META_HEADER:
        txt = _header(id2meta[did]) + "\n" + txt
    return (query, txt)

# 로컬 리랭커 (CrossEncoder)
def rerank_local(model_key: str, query: str, candidate_ids: list[str], topk: int = TOPK_FINAL):
    if model_key not in LOCAL_RERANKERS:
        raise KeyError(f"LOCAL_RERANKERS에 '{model_key}'가 없습니다. 등록된 키: {list(LOCAL_RERANKERS.keys())}")
    model_name = LOCAL_RERANKERS[model_key]
    ce = CrossEncoder(model_name, max_length=MAX_LEN_CE)
    scores = ce.predict([_pair(query, did) for did in candidate_ids])
    order  = np.argsort(-scores)
    ids_   = [candidate_ids[i] for i in order[:topk]]
    scs_   = [float(scores[i]) for i in order[:topk]]
    return ids_, scs_

# Cohere 리랭커 (API 필요)
COHERE_KEY = os.getenv("COHERE_KEY")
co_client = cohere.Client(COHERE_KEY) if (COHERE_KEY and 'co_client' not in globals()) else (co_client if 'co_client' in globals() else None)

def rerank_cohere(query: str, candidate_ids: list[str], topk: int = TOPK_FINAL, model: str = "rerank-multilingual-v3.0"):
    if not co_client:
        raise RuntimeError("COHERE_KEY가 설정되지 않았습니다.")
    docs = []
    for did in candidate_ids:
        txt = id2text[did]
        if USE_META_HEADER:
            txt = _header(id2meta[did]) + "\n" + txt
        docs.append(txt)
    resp = co_client.rerank(model=model, query=query, documents=docs, top_n=topk)
    ids_ = [candidate_ids[r.index] for r in resp.results]
    scs_ = [float(r.relevance_score) for r in resp.results]
    return ids_, scs_


#**쿼리 임베딩 차원과 인덱스 차원 일치 여부 확인**

In [13]:
print("문서 임베딩 shape:", embs.shape)
print("FAISS index dim:", index.d, " / ntotal:", index.ntotal)

# 쿼리 임베딩 한 번 찍어보기
tmp_q = "진단용 쿼리"
qv = e5_query_embed(tmp_q)
print("쿼리 임베딩 dim:", qv.shape[0])

# k가 인덱스 개수보다 큰지도 체크
print("요청 k=10, 실제 ntotal:", index.ntotal)

문서 임베딩 shape: (1057, 1024)
FAISS index dim: 1024  / ntotal: 1057
쿼리 임베딩 dim: 768
요청 k=10, 실제 ntotal: 1057


#**쿼리 인코더 자동 설정 모듈**
## - 인덱스의 임베딩 차원에 맞춰 적절한 쿼리 인코더를 자동으로 선택하고, 로드하며, 쿼리 벡터를 생성할 때 차원 불일치 방지

In [16]:
# 쿼리 인코더: 인덱스 차원에 자동 맞춤 (기존 e5_query_embed를 덮어씀)
from sentence_transformers import SentenceTransformer
import torch, numpy as np

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
_QE = None
_QE_NAME = None

# 인덱스 차원 → 권장 모델 매핑 (문서 임베딩 만든 모델로 바꾸면 가장 확실)
DIM2MODEL = {
    384:  "sentence-transformers/all-MiniLM-L6-v2",
    512:  "BAAI/bge-small-en-v1.5",
    768:  "intfloat/multilingual-e5-base",
    1024: "intfloat/multilingual-e5-large",  # 문서가 large면 여기로 자동 매칭
}

def configure_query_encoder(doc_dim: int | None = None, force_model: str | None = None):
    """index.d(문서 임베딩 차원)에 맞춰 쿼리 인코더 로드. force_model로 강제 지정 가능."""
    global _QE, _QE_NAME
    dim = int(index.d if doc_dim is None else doc_dim)
    name = force_model or DIM2MODEL.get(dim, "intfloat/multilingual-e5-large")
    if (_QE is None) or (_QE_NAME != name):
        _QE = SentenceTransformer(name, device=DEVICE)
        _QE.max_seq_length = 512
        _QE_NAME = name
        print(f"[query encoder] -> {name} (dim={_QE.get_sentence_embedding_dimension()})")
    assert _QE.get_sentence_embedding_dimension() == index.d, \
        f"쿼리 차원({_QE.get_sentence_embedding_dimension()}) ≠ 인덱스 차원({index.d})"

def e5_query_embed(q: str) -> np.ndarray:
    """인덱스 차원에 맞는 인코더로 쿼리 임베딩 생성 (기존 함수를 덮어씀)."""
    if _QE is None:
        configure_query_encoder()  # index.d 기준 자동 설정
    vec = _QE.encode([f"query: {q.strip()}"], normalize_embeddings=True, show_progress_bar=False)[0]
    return vec.astype("float32")


#**FAISS 인덱스에서 후보 문서를 검색하는 함수**
## - 인덱스가 비어있는지 확인
## - 쿼리 임베딩 차원과 인덱스 차원 일치 여부 검증
## - top-k 값이 유효한지 검사

In [17]:
# Dense 검색: 안전 가드 버전
def retrieve_candidates(query: str, topk: int = TOPK_RETR):
    if index.ntotal == 0:
        raise RuntimeError("FAISS index가 비어 있습니다 (ntotal=0).")
    qv = e5_query_embed(query)
    if qv.shape[0] != index.d:
        raise ValueError(f"쿼리 차원({qv.shape[0]}) ≠ 인덱스 차원({index.d})")
    k = min(int(topk), index.ntotal)
    if k <= 0:
        raise ValueError(f"요청 topk={topk}, but ntotal={index.ntotal}")

    qn = (qv / (np.linalg.norm(qv) + 1e-12)).astype("float32")[None, :]
    D, I = index.search(qn, k)
    cand_ids  = [ids[i] for i in I[0]]
    cand_scos = [float(s) for s in D[0]]
    return cand_ids, cand_scos


#**스모크 테스트(여러 리랭커가 실제로 작동되는지 확인)**
## - 리트리버가 반환한 top 10과 리랭커가 선정한 top 5 비교
## - top1_changed=True : 리트리버로 반환한 문서가 리랭킹 후 1등 문서가 바뀌었다는 의미

In [19]:
# 스모크 테스트: 여러 리랭커가 실제로 도는지 확인
import time

def _show_topk(ids, scores=None, k=5, show_text=True, max_chars=140):
    k = min(k, len(ids))
    for i in range(k):
        did = ids[i]
        sc  = scores[i] if (scores and i < len(scores)) else None
        meta = id2meta.get(did, {})
        line = f"{i+1:>2}. {did}" + (f" | score={sc:.4f}" if sc is not None else "")
        print(line)
        print(f"    meta: file={meta.get('filename')} chunk={meta.get('chunk_index')}")
        if show_text:
            txt = (id2text.get(did,"") or "").replace("\n"," ")
            print("    text:", txt[:max_chars] + ("..." if len(txt) > max_chars else ""))

def test_one_model(query: str, model_key: str, retr_topk=10, rank_topk=5, method="local", show_ctx=True):
    print("="*80)
    print(f"[TEST] method={method} model={model_key}")
    # 1) 후보 생성
    t0 = time.time()
    cand_ids, cand_scores = retrieve_candidates(query, topk=retr_topk)
    t1 = time.time()
    print(f"retrieved={len(cand_ids)} in {t1-t0:.3f}s")
    # ⬇️ 리트리버 Top-10 전부 출력
    _show_topk(cand_ids, cand_scores, k=retr_topk, show_text=show_ctx)

    # 2) 리랭킹
    t2 = time.time()
    if method == "local":
        ids_, scs_ = rerank_local(model_key, query, cand_ids, topk=rank_topk)
    elif method == "cohere":
        ids_, scs_ = rerank_cohere(query, cand_ids, topk=rank_topk, model=model_key)
    else:
        raise ValueError("method must be 'local' or 'cohere'")
    t3 = time.time()

    changed_top1 = (ids_ and cand_ids and ids_[0] != cand_ids[0])
    print(f"reranked={len(ids_)} in {t3-t2:.3f}s | top1_changed={changed_top1}")
    # ⬇️ 리랭커 Top-5 전부 출력 (len(ids_) == rank_topk)
    _show_topk(ids_, scs_, k=len(ids_), show_text=show_ctx)

# 실행 예시
q = "종합부동산세 과세 대상은 무엇인가?"
for m in ["MiniLM-msmarco", "bge-reranker-v2-m3", "distilroberta-msmarco", "tinybert-msmarco"]:
    try:
        test_one_model(q, m, retr_topk=10, rank_topk=5, method="local", show_ctx=True)
    except Exception as e:
        print(f"[ERROR:{m}]", e)

# Cohere
if co_client:
    try:
        test_one_model(q, "rerank-multilingual-v3.0", retr_topk=10, rank_topk=5, method="cohere", show_ctx=True)
    except Exception as e:
        print("[ERROR:cohere]", e)
else:
    print("[SKIP] Cohere: COHERE_KEY 미설정")

[TEST] method=local model=MiniLM-msmarco
retrieved=10 in 0.028s
 1. 판례_193173#4 | score=0.8412
    meta: file=판례_193173.txt chunk=4
    text: 및 종합부동산세의 과세대상이 되는 주택을 ‘세대의 세대원이 장기간 독립된 주거생활을 영위할 수 있는 구조로 된 건축물의 전부 또는 일부 및 그 부속토지’로 규정하고 있다. 한편 구 종합부동산세법 제2조 제5호 는 ‘주택분 재산세라 함은 지방세법 제18...
 2. 판례_71063#4 | score=0.8400
    meta: file=판례_71063.txt chunk=4
    text: 부할 의무가 있다. 제8조 (과세표준) ① 주택에 대한 종합부동산세의 과세표준은 납세의무자별로 주택의 공시가격을 합산한 금액에서 6억 원을 공제한 금액으로 한다. 다만, 그 금액이 영보다 작은 경우에는 영으로 본다. ② 다음 각 호의 어느 하나에 해당...
 3. 판례_237251#6 | score=0.8383
    meta: file=판례_237251.txt chunk=6
    text: 종합부동산세 과세표준에 대하여 재산세가 중복 부과되는 부분은 여전히 (B ＋ C)로서 개정 전과 동일하다. D 부분은 애초부터 종합부동산세는 물론 재산세도 부과되지 않는 영역이기 때문이다. (3) 2008. 12. 26. 개정 전후의 종합부동산세 과세...
 4. 판례_158078#2 | score=0.8363
    meta: file=판례_158078.txt chunk=2
    text: 고 있지 아니한 점, 개정된 구 종합부동산세법 시행령(2009. 4. 21. 대통령령 제21432호로 개정되기 전의 것) 제7조 제2항 은 전년도 재산세액상당액을 ‘해당 연도의 별도합산과세토지에 대하여 직전 연도의 지방세법( 같은 법 제188조 제3항...
 5. 판례_237299#3 | score